来源书籍：Natural-Language-Processing-with-TensorFlow

## 导入库

In [4]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import logging
%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger("tensorflow").setLevel(logging.ERROR)

## 通过placeholder传递输入
sigmoid实例

In [11]:
# 定义graph和session
graph = tf.Graph() # 创建图
session = tf.InteractiveSession(graph=graph) # 创建会话

# 构建图
# 创建占位符placeholder
x=tf.placeholder(shape=[1,10],dtype=tf.float32,name='x')
# 变量 W
W=tf.Variable(tf.random_uniform(shape=[10,5],minval=-0.1,maxval=0.1,dtype=tf.float32),name='W')
# 变量 b
b=tf.Variable(tf.zeros(shape=[5],dtype=tf.float32),name='b')

h=tf.nn.sigmoid(tf.matmul(x,W)+b) # sigmoid运算
#
tf.global_variables_initializer().run() # 初始化全局变量

# 传给x值
h_eval=session.run(h,feed_dict={x:np.random.rand(1,10)})
print(h_eval)
session.close()

[[0.4941275  0.51278746 0.49341637 0.5056394  0.50781184]]


## 通过TensorFlow 张量传递输入

In [10]:
# 定义图
graph=tf.Graph()
session=tf.InteractiveSession(graph=graph)

# 构建图

# 声明x并赋值,
x=tf.constant(value=[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]],dtype=tf.float32,name='x')

# 变量 W
W=tf.Variable(tf.random_uniform(shape=[10,5],minval=-0.1,maxval=0.1,dtype=tf.float32),name='W')
# 变量 b
b=tf.Variable(tf.zeros(shape=[5],dtype=tf.float32),name='b')

h=tf.nn.sigmoid(tf.matmul(x,W)+b) # 计算

# 初始化变量
tf.global_variables_initializer().run()

# 直接计算
h_eval=session.run(h)
print(h_eval)

# 关闭会话
session.close()

[[0.5038708  0.51638865 0.48006743 0.49199152 0.461249  ]]


## 创建 Tensorflow Pipeline

In [14]:
# 定义图和会话
graph=tf.Graph()
session=tf.InteractiveSession(graph=graph)

# 文件
filenames=['data/19/test%d.txt' %i for i in range(1,4)]
filename_queue=tf.train.string_input_producer(filenames,capacity=3,shuffle=True,name='string_input_producer')

In [15]:
# 检查文件是否存在
for f in filenames:
    if not tf.gfile.Exists(f):
        raise ValueError('Failed to find file: ' + f)
    else:
        print('File %s found.'%f)

File data/19/test1.txt found.
File data/19/test2.txt found.
File data/19/test3.txt found.


In [18]:
reader=tf.TextLineReader()
key, value = reader.read(filename_queue, name='text_read_op')

# 默认值
record_defaults = [[-1.0], [-1.0], [-1.0], [-1.0], [-1.0], [-1.0], [-1.0], [-1.0], [-1.0], [-1.0]]
col1, col2, col3, col4, col5, col6, col7, col8, col9, col10 = tf.decode_csv(value, record_defaults=record_defaults)
features = tf.stack([col1, col2, col3, col4, col5, col6, col7, col8, col9, col10])

x = tf.train.shuffle_batch([features], batch_size=3,
                           capacity=5, name='data_batch', 
                           min_after_dequeue=1,num_threads=1)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=session)

In [21]:
W = tf.Variable(tf.random_uniform(shape=[10,5], minval=-0.1, maxval=0.1, dtype=tf.float32),name='W') 
b = tf.Variable(tf.zeros(shape=[5],dtype=tf.float32),name='b') 
h = tf.nn.sigmoid(tf.matmul(x,W) + b)
tf.global_variables_initializer().run()
for step in range(5):
    x_eval, h_eval = session.run([x,h]) 
    print('========== Step %d =========='%step)
    print('Evaluated data (x)')
    print(x_eval)
    print('Evaluated data (h)')
    print(h_eval)
    print('')

coord.request_stop()
coord.join(threads)
session.close()

========== Step 0 ==========
Evaluated data (x)
[[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Evaluated data (h)
[[0.5049817  0.49255627 0.5058895  0.50510323 0.50227076]
 [0.5049817  0.49255627 0.5058895  0.50510323 0.5022707 ]
 [0.50498176 0.4925563  0.5058895  0.50510323 0.5022707 ]]

========== Step 1 ==========
Evaluated data (x)
[[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
 [1.  0.9 0.8 0.7 0.6 0.5 0.4 0.3 0.2 0.1]
 [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
Evaluated data (h)
[[0.5049817  0.49255627 0.5058895  0.50510323 0.50227076]
 [0.5438934  0.44402477 0.5267701  0.5240376  0.50090057]
 [0.50498176 0.4925563  0.5058895  0.50510323 0.5022707 ]]

========== Step 2 ==========
Evaluated data (x)
[[1.  0.9 0.8 0.7 0.6 0.5 0.4 0.3 0.2 0.1]
 [1.  0.9 0.8 0.7 0.6 0.5 0.4 0.3 0.2 0.1]
 [1.  0.9 0.8 0.7 0.6 0.5 0.4 0.3 0.2 0.1]]
Evaluated data (h)
[[0.5438934  0.44402477 0.5267701  0.52403754 0.50090057]

## TF实现损失函数
- MSE
- Cross Entropy Loss

In [25]:
# 定义图和会话
graph=tf.Graph()
session=tf.InteractiveSession(graph=graph)

x=tf.constant([[2,4],[6,8]],dtype=tf.float32)
x_hat=tf.constant([[3,5],[7,9]],dtype=tf.float32)
# MSE=（1**2+1**2+1**2+1**2）/2
MSE=tf.nn.l2_loss(x_hat-x)

# cross_entropy with logits 
y=tf.constant([[1,0],[0,1]],dtype=tf.float32)
y_hat=tf.constant([[3,1],[2,5]],dtype=tf.float32)

CE=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_hat,labels=y))

print('MSE Loss: ',session.run(MSE))
print('CE Loss: ',session.run(CE))
session.close()

MSE Loss:  2.0
CE Loss:  0.08775769
